---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [203]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_curve, accuracy_score

## Read in Data

In [240]:
train = pd.read_csv("train.csv", encoding = "ISO-8859-1")
test = pd.read_csv("test.csv", encoding = "ISO-8859-1")
addresses = pd.read_csv("addresses.csv")
latlons = pd.read_csv("latlons.csv")

<string>:2: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 2.95 s, sys: 196 ms, total: 3.15 s
Wall time: 4.57 s


## Preprocessing Steps
1. Get the lat/lon features 
2. drop NA-compliance rows from training data
    * also fillna for lon/lat data
3. Find columns to drop:
    * take difference between columns in train and test sets
    * ticket_id is not a feature
    * drop address columns
4. Alternatively, columns to keep:
    * agency_name
    * inspector_name
    * judgment_amount
    * violation_code
    * disposition
    * lat
    * lon
    * possibly generate new column which is difference between ticket_issued_date and hearing_date
5. convert appropriate columns to categorical

## Fit Gradient Boosting Classifiers and Evaluate

In [253]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# clf = GradientBoostingClassifier().fit(X_train, y_train)
# pred_probas = clf.predict_proba(X_test)[:, 1]

# # Calculate Area Under the (Receiver Operating Characteristic) Curve
# fpr, tpr, threshold = roc_curve(y_test, pred_probas)
# roc_auc = auc(fpr, tpr)
# print(roc_auc)

In [259]:
# %%time
# ## CV Score
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(clf, X_train, y_train, scoring="roc_auc", cv=5)
# print(scores)
# print(scores.mean()) # cross-validated score

[ 0.80912058  0.79372945  0.80902666  0.80836182  0.80674437]
0.805396574587
CPU times: user 33.9 s, sys: 8 ms, total: 33.9 s
Wall time: 48.3 s


## Submission

In [269]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_curve, accuracy_score
from sklearn.preprocessing import LabelEncoder

def blight_model():
    ## Part 1 - Fit Model with Training Data
    # 1. Get lat/lon features
    full_address = pd.merge(addresses, latlons, on='address', how='left') # Join latlons to addresses
    train_data = pd.merge(train, full_address, on='ticket_id', how='inner') # Join to train dataframe
    # 2. drop rows where compliance = NaN (also lat and lon)
    train_data = train_data.dropna(subset=['compliance'])
    train_data['lat'].fillna(method='pad', inplace=True)
    train_data['lon'].fillna(method='pad', inplace=True)      
    train_data['compliance'] = train_data['compliance'].astype(int) # convert compliance data to int
    # 3. columns to keep
    train_keep_cols = ['agency_name', 'inspector_name', 'judgment_amount', 'violation_code', 
                       'disposition', 'lat', 'lon']
    # 4. convert columns to categorical
    le = LabelEncoder()
    train_data['agency_name'] = le.fit_transform(train_data["agency_name"])
    train_data['inspector_name'] = le.fit_transform(train_data["inspector_name"])
    train_data['violation_code'] = le.fit_transform(train_data["violation_code"])
    train_data['disposition'] = le.fit_transform(train_data["disposition"])
    # 5. Fit Gradient Boosting Classifier - don't need train/test split -> fit on all data this time
    X = train_data[train_keep_cols]
    y = train_data['compliance']
    clf = GradientBoostingClassifier().fit(X, y)
    
    ## Part 2 - Prepare Test Data for Submission
    # 1. Get lat/lon features
    full_address = pd.merge(addresses, latlons, on='address', how='left') # Join latlons to addresses
    test_data = pd.merge(test, full_address, on='ticket_id', how='inner') # Join to test dataframe
    # 2. drop NA rows for lat and lon - s
    test_data['lat'].fillna(method='pad', inplace=True)
    test_data['lon'].fillna(method='pad', inplace=True)  
    # 3. columns to keep
    test_keep_cols = ['agency_name', 'inspector_name', 'judgment_amount', 'violation_code', 
                       'disposition', 'lat', 'lon']
    # 4. convert columns to categorical
    le = LabelEncoder()
    test_data['agency_name'] = le.fit_transform(test_data["agency_name"])
    test_data['inspector_name'] = le.fit_transform(test_data["inspector_name"])
    test_data['violation_code'] = le.fit_transform(test_data["violation_code"])
    test_data['disposition'] = le.fit_transform(test_data["disposition"])
    X = test_data[test_keep_cols]

    
    ## Make predictions and Reindex
    test_proba = clf.predict_proba(X)[:, 1]
    test_data['compliance'] = test_proba
    test_data.set_index('ticket_id', inplace=True)
    
    return test_data.compliance # Your answer here

In [267]:
# blight_model()

ticket_id
284932    0.342492
285362    0.096986
285361    0.225118
285338    0.416017
285346    0.386402
285345    0.416017
285347    0.427940
285342    0.541657
285530    0.116598
284989    0.186794
285344    0.448606
285343    0.159026
285340    0.163185
285341    0.456231
285349    0.386402
285348    0.416017
284991    0.186794
285532    0.160949
285406    0.139504
285001    0.253624
285006    0.194619
285405    0.108937
285337    0.140345
285496    0.334804
285497    0.334396
285378    0.098261
285589    0.149891
285585    0.317094
285501    0.237405
285581    0.098261
            ...   
376367    0.143125
376366    0.176434
376362    0.176434
376363    0.323167
376365    0.143125
376364    0.176434
376228    0.275419
376265    0.171522
376286    0.871759
376320    0.187015
376314    0.180828
376327    0.925590
376385    0.883347
376435    0.282678
376370    0.918445
376434    0.312475
376459    0.344061
376478    0.075431
376473    0.183999
376484    0.293906
376482    0.099299
37

In [270]:
# bm = blight_model()
# res = 'Data type Test: '
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
# res += 'Data shape Test: '
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
# res += 'Data Values Test: '
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
# res += 'Data Values type Test: '
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
# res += 'Index type Test: '
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
# res += 'Index values type Test: '
# res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

# res += 'Output index shape test:'
# res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

# res += 'Output index test: '
# if bm.index.shape==(61001,):
#     res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
# else:
#     res+='Failed'
# print(res)

Data type Test: Passed
Data shape Test: Passed
Data Values Test: Passed
Data Values type Test: Passed
Index type Test: Passed
Index values type Test: Passed
Output index shape test:Passed
Output index test: Passed

